In [ ]:
!git clone https://github.com/kk-digital/kcg-ml-sd1p4
%cd kcg-ml-sd1p4

In [ ]:
!pip install -r requirements.txt

In [9]:
import sys
import os
import shutil
import re
import zipfile
import json
from PIL import Image
import requests
import clip
import numpy as np
from transformers import CLIPModel, CLIPImageProcessor
import torch

sys.path.insert(0, os.getcwd())
from utility.clip.clip_feature_zip_loader import ClipFeatureZipLoader
from utility.clip.clip import ClipModel


In [ ]:
def test_clip_skip(layer_number=None):
    clip_model = 'openai/clip-vit-large-patch14'
    model = CLIPModel.from_pretrained(clip_model)

    processor = CLIPImageProcessor.from_pretrained(clip_model)
    url = "http://images.cocodataset.org/val2017/000000039769.jpg"
    image = Image.open(requests.get(url, stream=True).raw)
    inputs = processor(images=image, return_tensors="pt")

    vision_outputs = model.vision_model(
        **inputs,
        output_hidden_states=True,
    )

    # Use the layer before the last one if no layer_number is provided
    if layer_number is None:
        layer_number = len(vision_outputs.hidden_states) - 2

    layer_output = vision_outputs.hidden_states[layer_number]
    pooled_output = layer_output[:, 0, :]
    pooled_output = model.vision_model.post_layernorm(pooled_output)
    image_features = model.visual_projection(pooled_output)
    print(image_features.shape)
    assert image_features.shape == torch.Size([1, 768])

# Use the function without specifying a layer; it will use the layer before the last
test_clip_skip()

# Or specify a layer when calling the function
#layer_number = 4
#test_clip_skip(layer_number)


In [ ]:
def test_clip_model_clip_skip():
    clip_model = ClipModel(verbose=True, clip_skip=True)
    clip_model.load_clip()

    # sample image
    url = "http://images.cocodataset.org/val2017/000000039769.jpg"
    image = Image.open(requests.get(url, stream=True).raw)

    image_features, penultimate_output = clip_model.get_image_features(image)
    print(image_features.shape)
    clip_model.unload_clip()

    assert image_features.shape == torch.Size([1, 768])

# Execute the function
test_clip_model_clip_skip()
